In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import itertools

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
print("Imported depending packages")

## 决策信息系统

- 条件属性
- 决策属性

## 导入数据集

通过 pandas 导入数据集

In [ ]:
dataset = pd.read_csv("../input/iris/Iris.csv")
dataset

## 使用 min-max 标准化

In [ ]:
dataset = (dataset - dataset.min()) / (dataset.max() - dataset.min())
dataset

## 使用二分法进行数据处理

简单的二分法，取当前 dataset 中每列中最大值和最小值相加的一半作为基准，大于基准的数据设为1，小于基准的数据设为0,同时将数据类型设置为int

In [ ]:
for column in dataset:
    max = dataset[column].max()
    min = dataset[column].min()
    base = (max + min)/2
    print(max, min, base)
    big = dataset[column] >= base
    small = dataset[column] < base
    dataset[column] = dataset[column].astype("int")
    dataset.loc[big, column] = 1
    dataset.loc[small, column] = 0
    
dataset

## 使用 One hot 处理数据

In [ ]:
for column in ["Species"]:
    oneHot = pd.get_dummies(dataset[column], prefix=column)
    dataset = dataset.join(oneHot)
    dataset = dataset.drop(column, axis = 1)
    
dataset

## 使用模拟退火进行多元优化

In [ ]:
big = dataset.iloc[:, 0] > 0.5
big

In [ ]:
# 损失函数

def lossFun(p):
    data = dataset.copy()
    for index, base in enumerate(p):
        big = data.iloc[:, index] >= base
        small = data.iloc[:, index] < base
        data.iloc[:, index] = data.iloc[:, index].astype("int")
        data.iloc[big, index] = 1
        data.iloc[small, index] = 0
#     print(data)
    rate = coordinatedRate(data)
    return 1 - rate
lossFun([1,2,3])

In [ ]:
import torch
from sko.GA import GA
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

ga = GA(
    func = lossFun,
    n_dim = dataset.shape[1],
    size_pop = 50,
    max_iter = 800,
    lb = np.zeros(dataset.shape[1], int),
    ub = np.ones(dataset.shape[1], int),
    precision = 1e-7
)

ga.to(device = device)
best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)

## 坐标下降法

In [ ]:
dataProcessed = dataset.copy()
# Doing some pre-processing for the dataset here

In [ ]:
baseList = np.ones(data.shape[1], int)*0.5
for column in baseList:
    max = 0
    maxBase = 0
    for base in np.arange(0.1,1,0.1):
        data = dataProcessed.copy()
        big = data[column] >= base
        small = data[column] < base
        data[column] = data[column].astype("int")
        data.loc[big, column] = 1
        data.loc[small, column] = 0
        rate = coordinatedRate(data)
        if rate > max:
            max = rate
            maxBase = base
    print("For ", column, " maxBase ", maxBase)
    print("==========")

In [ ]:

np.ones(dataset.shape[1], int)*0.5

In [ ]:
print(np.arange(0.1,1,0.1))

## 判断是否粒协调

In [ ]:
def isCoordinated(row, attrdataset, decidataset):
    xaa = []
    xdd = []
#     print(attrdataset)
#     print(decidataset)
    for index in attrdataset.index:
        isAddxaa = True
        for column in attrdataset:
            if attrdataset.loc[index, column] == 0 and attrdataset.loc[row, column] == 1:
                isAddxaa = False
        if isAddxaa:
            xaa.append(index)
#     print(xaa)
        
    for index in decidataset.index:
        isAddxdd = True
        for column in decidataset:
            if decidataset.loc[index, column] == 0 and decidataset.loc[row, column] == 1:
                isAddxdd = False
        if isAddxdd:
            xdd.append(index)
#     print(xdd)  
    if contain(xdd, xaa):
        return True
    return False
            
    
def contain(array1, array2):
    for x in array2:
        if x not in array1:
            return False
    return True

print(isCoordinated(1, dataset.iloc[:, 1:5], dataset.iloc[:, 5:-1]))



## 计算粒协调的比例

In [ ]:
def coordinatedRate(dataset):
    trueNum = 0
    falseNum = 0
    isTrue = []
    for index in dataset.index:
        if isCoordinated(index, dataset.iloc[:, 1:-3], dataset.iloc[:, -3:]):
            trueNum += 1
            isTrue.append(True)
        else:
            falseNum += 1
            isTrue.append(False)
    rate = trueNum / (trueNum + falseNum)
    return rate
print(coordinatedRate(dataset))
# print(isTrue)
# coordinatedDataset = dataset[]

In [ ]:
coordinatedDataset = dataset[isTrue]
coordinatedDataset

In [ ]:
for column in dataset:
    print(column)

## 粒协调决策形式背景的属性约简

> 「粒协调决策形式背景的属性约简与规则融合」算法 1

- Input：*F=(U, A, I, D, G)*
- Output：*B*

### 算法

1. 已知 *F* 为粒协调形式背景
2. 构造 *F* 的辨识矩阵
3. 提取辨识矩阵中的所有集合
4. 去重并删除空集
5. 删除存在包含关系的集合
6. 对剩余集合求笛卡尔积
7. 输出 *B* 即为该形式背景的粒约简

## 例子

对于 *F=(U, A, I, D, G)*

| **U** | a1 | a2 | a3 | a4 | a5 | a6 | a7 | a8 | d |
|---|----|----|----|----|----|----|----|----|---|
| **x1** | 1 | − | − | − | − | 1 | − | − | 1 |
| **x2** | 1 | − | − | − | − | 1 | 1 | - | 1 |
| **x3** | 1 | 1 | − | − | − | 1 | 1 | - | 1 |
| **x4** | - | 1 | − | − | 1 | 1 | 1 | 1 | 3 |
| **x5** | 1 | − | 1 | − | 1 | - | - | - | 2 |
| **x6** | 1 | 1 | 1 | − | 1 | - | - | - | 2 |
| **x7** | - | 1 | 1 | 1 | - | - | - | - | 3 |

*U = {x1, x2, x3, x4, x5, x6, x7}*

*A = {a1,a2,a3,a4,a5,a6,a7,a8}*

In [ ]:
# 初始化上面的例子

u = np.array(["x1", "x2", "x3", "x4", "x5", "x6", "x7"])
a = np.array(["a1", "a2", "a3", "a4", "a5", "a6", "a7", "a8"])
d = np.array(["d"])
i = np.array([[1, 0, 0, 0, 0, 1, 0, 0, 1],
              [1, 0, 0, 0, 0, 1, 1, 0, 1],
              [1, 1, 0, 0, 0, 1, 1, 0, 1],
              [0, 1, 0, 0, 1, 1, 1, 1, 3],
              [1, 0, 1, 0, 1, 0, 0, 0, 2],
              [1, 1, 1, 0, 1, 0, 0, 0, 2],
              [0, 1, 1, 1, 0, 0, 0, 0, 3],
             ])
print(u,a,d,i)

In [ ]:
# 构造辨识矩阵
discernMat = []
for xa in range(0, len(u)):
    dRow = []
    for xb in range(0, len(u)):
        dBlock = []
        for attr in range(0, len(a)):
            if i[xa][attr] == 1 and i[xb][attr] != 1 and i[xb][8] != i[xa][8]:
                dBlock.append(a[attr])
        dRow.append(dBlock)
    discernMat.append(dRow)

print(discernMat)
dMat = discernMat

def contain(array1, array2):
    for x in array2:
        if x not in array1:
            return False
    return True

dList = []
for y in dMat:
    for x in y:
        if x not in dList:
            dList.append(x)
dList.remove([])
print()
print(dList)

removeList = []
for aList in dList:
    for aListCompare in dList:
        if aListCompare != aList:
            if contain(aListCompare,aList):
                if aListCompare not in removeList:
                    removeList.append(aListCompare)
for q in removeList:
    dList.remove(q)
print(dList)
print("粒约简：")
for item in itertools.product(*dList):
    print(item)
    
    